Lista de variaveis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def categorize(value):
    if value == 'nao': 
        return 0 
    return 1


dataset = pd.read_csv("callcenter_marketing.csv") #importação da planilha

dataset['resultado']=dataset['resultado'].apply(categorize) #modificacão da coluna de resultado


#dataset campos numericos
dataset_numerico = dataset[['idade', 'duracao', 'qtd_contatos_campanha', 'dias_ultimo_contato', 'qtd_contatos_total', 'indice_precos_consumidor', 'indice_confianca_consumidor','euribor3m']]

#dataset campos categoricos
dataset_categorico = dataset[['profissao', 'estado_civil', 'educacao', 'inadimplente', 'emprestimo_moradia', 'emprestimo_pessoal', 'meio_contato', 'mes', 'dia_da_semana', 'campanha_anterior']]

#campos categoricos
for nome_coluna in dataset_categorico.columns:
    dataset_categorico[nome_coluna].fillna('NÃO INFORMADA', inplace=True)
    
#dummy dos campos categoricos
dataset_categorico_corrigido = pd.get_dummies(dataset_categorico)



Plotagem com a correção das variaveis

In [353]:
#ig, ax = plt.subplots(figsize=(10,10))

dataset_concat_com_valor= pd.concat([dataset_categorico_corrigido,dataset['resultado']],axis=1)

#del dataset_concat_com_valor['campanha_anterior_fracasso']
#sns.heatmap(dataset_concat_com_valor.corr())#, annot=True, linewidths=.5, ax=ax)



Concatenação do dataset categorico ja corrigido com o dataset numerico

In [354]:

dataset_concat = pd.concat([dataset_categorico_corrigido, dataset_numerico], axis = 1)
#dataset_concat.head()

In [355]:
#target
y = dataset['resultado']
X = dataset_concat.iloc[:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


Modelo linear

In [356]:
from sklearn.linear_model import LinearRegression

def categorizeNew(value):
    if value <= 0.5: 
        return 0 # not admitted
    return 1 # admitted

lr = LinearRegression() # criação do modelo
lr.fit(X_train,y_train)

previsto = lr.predict(X_test)

serie_prevista = pd.Series(previsto) 
serie_prevista = serie_prevista.apply(categorizeNew)

print('O acerto percentual é: '+
str((
np.count_nonzero(serie_prevista.values==y_test)
/serie_prevista.size)*100))


O acerto percentual é: 90.02184996358339


In [357]:
# regressao logística

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs')
logreg.fit(X_train, y_train )
acuracia_logistica = logreg.score(X_test, y_test)
display(logreg.score(X_test, y_test))


0.8876750020231448

In [358]:
#decision tree

from sklearn import tree
clf_dt = tree.DecisionTreeClassifier()
clf_dt = clf_dt.fit(X_train, y_train)

#predição
y_pred = clf_dt.predict(X_test)
acuracia_dt = clf_dt.score(X_test,y_test)
acuracia_dt


0.8905883305009307